In [182]:
import igl # Library to load meshes and perform operations on them
import meshplot as mp # Library to visualize meshes and point clouds
import vedo as vd # Library to visualize meshes and point clouds
import polyscope as ps # Library to visualize meshes
import numpy as np # Library to perform operations on matrices
import os # Library to perform operations on files and directories
import matplotlib.pyplot as plt

# Importing the classes and functions from the visualization folder
vd.settings.default_backend = 'k3d'

# Directory path
dir_path = os.getcwd()

In [183]:
v, f = igl.read_triangle_mesh("C:/Users/aikyna/Desktop/hananLab/hJupyter/models/Hall.obj")
V = len(v) # Number of vertices
F = len(f) # Number of faces

v1, v2, k1, k2 = igl.principal_curvature(v, f)
normals = igl.per_vertex_normals(v, f)
h2 = 0.5 * (k1 + k2)

rads_min, rads_max = np.zeros(V), np.zeros(V)

for v_ind in range(V):
    rads_min[v_ind] = 1 / abs(k1[v_ind])
    rads_max[v_ind] = 1 / abs(k2[v_ind])
    if (rads_min[v_ind] > rads_max[v_ind]):
        rads_min[v_ind], rads_max[v_ind] = rads_max[v_ind], rads_min[v_ind]
        
rads = np.zeros(V)
for v_ind in range(V):
    rads[v_ind] = 1.0 / abs(h2[v_ind])

centers = np.zeros((V, 3))
for v_ind in range(V):
    centers[v_ind] = v[v_ind] + normals[v_ind] / h2[v_ind]

In [184]:
center_abs_init = np.zeros(3)

matrix_s = np.zeros((3 * V, 3))
vector_b = np.zeros(3*V)
for v_ind in range(V):
    normal = normals[v_ind]
    matrix_s[3 * v_ind, :] = np.array([0, -normal[2], normal[1]])
    matrix_s[3 * v_ind + 1, :] = np.array([normal[2], 0, -normal[0]])
    matrix_s[3 * v_ind + 2, :] = np.array([-normal[1], normal[0], 0])
    vector_b[3 * v_ind:3 * v_ind + 3] = matrix_s[3 * v_ind:3 * v_ind + 3, :] @ v[v_ind]
#print(matrix_s.T@matrix_s)
#print(matrix_s.T@vector_b)

center_abs_init = np.linalg.lstsq(matrix_s.T@matrix_s, matrix_s.T@vector_b, rcond=None)[0]
print(center_abs_init)

[18.97791031 26.11760639 -4.87671118]


In [185]:
vector = np.linalg.lstsq(normals, np.ones(V), rcond=None)[0]
vector = vector/np.linalg.norm(vector)
print(vector)

[-0.0101157  -0.56749374  0.82331557]


In [186]:
p = mp.plot(v, f, return_plot=True);
p.add_lines(v, v + normals, shading={"line_color": "red"});
p.add_points(np.array([center_abs_init]), shading={"point_color": "green", "point_size": 10.0});


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(19.867670…

In [187]:
J = np.zeros((V, 3))
r = np.zeros(V)
X_0 = center_abs_init

p_length = np.zeros(V)
for v_ind in range(V):
    p_length[v_ind] = np.linalg.norm(center_abs_init - v[v_ind])

w = np.zeros(V)
for v_ind in range(V):
    angle = np.arccos(np.dot(v[v_ind] - center_abs_init, normals[v_ind]) / np.linalg.norm(v[v_ind] - center_abs_init))
    w[v_ind] = np.tan(angle / 2)

In [188]:
def compute_J(v, normals, w, X, p_length):
    center_var = X
        
    for v_ind in range(V):
        J[v_ind, 0:3] = -normals[v_ind] / p_length[v_ind] * w[v_ind]
        
        r[v_ind] = w[v_ind] * (np.dot(v[v_ind] - center_var, normals[v_ind]) / p_length[v_ind] - 1)

        p_length[v_ind] = np.linalg.norm(v[v_ind] - center_var)
        angle = np.arccos(np.dot(v[v_ind] - center_var, normals[v_ind]) / np.linalg.norm(v[v_ind] - center_var))
        w[v_ind] = np.tan(angle / 2)

In [189]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, linalg
from math import pi
# Compute pseudo Hessian
X = X_0

for i in range(10):
    compute_J(v, normals, w, X, p_length)    

    H = J.T@J

    H[np.diag_indices_from(H)] += np.diag(H).max()*1e-6

    # Sparse matrix H
    H = csc_matrix(H)

    # Solve for dx
    dx = linalg.spsolve(H, -J.T@r)
    # Update vertices
    X = X + 0.9*dx
    
    # energy
    energy = r.T@r
    print(f"energy: {energy}\t dx: {np.linalg.norm(dx)}")
print(X, np.linalg.norm(X))
print(np.arccos(np.dot(X - center_abs_init, -vector) / np.linalg.norm(X - center_abs_init))/ pi * 180)
opt_point = X


energy: 19.41267031184627	 dx: 8.271167236282116
energy: 5.9859316874521165	 dx: 0.827164276634352
energy: 7.38177016821475	 dx: 6.765561187850496
energy: 1.794248082672338	 dx: 1.4273949995103992
energy: 7.7425683602200905	 dx: 7.121430418560399
energy: 2.2726177321466867	 dx: 2.3799670665597983
energy: 9.865911908826401	 dx: 9.175159798083364
energy: 3.897890258882365	 dx: 4.103012730889697
energy: 14.978501414274767	 dx: 13.326950932962099
energy: 7.449752776467335	 dx: 7.3426660786862215
[ 23.28116508  67.5718784  -39.43971299] 81.63003343624057
16.08329894794753


In [190]:
p = mp.plot(v, f, return_plot=True);
p.add_lines(v, v + normals, shading={"line_color": "red"});
p.add_points(np.array([center_abs_init, opt_point]), shading={"point_color": "green", "point_size": 10.0});

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(19.867670…

In [199]:
sum1 = 0
sum2 = 0
for v_ind in range(V):
    angle1 = np.arccos(np.dot(normals[v_ind], v[v_ind] - center_abs_init) / np.linalg.norm(v[v_ind] - center_abs_init))
    angle2 = np.arccos(np.dot(normals[v_ind], v[v_ind] - opt_point) / np.linalg.norm(v[v_ind] - opt_point))
    sum1 += angle1 ** 2
    sum2 += angle2 ** 2
    if (angle1 < 0 or angle2 < 0):
        print(angle1, angle2)
print(sum1, sum2)


1531.5472372718434 2292.5899947005505


In [192]:
center_abs = X

In [193]:
rad_init_min = abs(np.linalg.norm(center_abs - centers[0]) - rads[0])
rad_init_max = np.linalg.norm(center_abs - centers[0]) + rads[0]
for v_ind in range(V):
    rad_init_min = max(rad_init_min, abs(np.linalg.norm(center_abs - centers[v_ind]) - rads[v_ind]))
    rad_init_max = min(rad_init_max, np.linalg.norm(center_abs - centers[v_ind]) + rads[v_ind])

print("Radius minimum = ", rad_init_min)
print("Radius maximum = ", rad_init_max)

rad_abs = (rad_init_max + rad_init_min) * 0.5

Radius minimum =  77.91907686604866
Radius maximum =  57.20825688783768


In [194]:
J = np.zeros((2 * V, 1 + 2 * V))
r = np.zeros(2 * V)
X_0 = np.concatenate(([rad_abs * rad_abs], np.random.rand(2*V)))

In [195]:
def compute_J(v, normals, center_abs, rads_min, rads_max, X):
    rad_var = X[0]
        
    for v_ind in range(V):
        mu_1 = X[2 * v_ind + 1]
        mu_2 = X[2 * v_ind + 2]

        J[v_ind, 0] = 1 / np.dot(center_abs - v[v_ind], normals[v_ind]) / 2
        J[v_ind, 2 * v_ind + 1] = 2 * mu_1
        
        J[V + v_ind, 0] = 1 / np.dot(center_abs - v[v_ind], normals[v_ind]) / 2
        J[V + v_ind, 2 * v_ind + 2] = -2 * mu_2
        
        r[v_ind] = (rad_var - np.dot(center_abs - v[v_ind], center_abs - v[v_ind])) / (2 * np.dot(center_abs - v[v_ind], normals[v_ind])) - rads_max[v_ind] + mu_1 ** 2
        r[V + v_ind] = (rad_var - np.dot(center_abs - v[v_ind], center_abs - v[v_ind])) / (2 * np.dot(center_abs - v[v_ind], normals[v_ind])) - rads_min[v_ind] - mu_2 ** 2
        

In [196]:
# from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, linalg

# # Compute pseudo Hessian
# X = X_0

# for i in range(15):
#     compute_J(v, normals, center_abs, rads_min, rads_max, X)    

#     H = J.T@J

#     H[np.diag_indices_from(H)] += np.diag(H).max()*1e-6

#     # Sparse matrix H
#     H = csc_matrix(H)

#     # Solve for dx
#     dx = linalg.spsolve(H, -J.T@r)
#     # Update vertices
#     X = X + 0.9*dx
    
#     # energy
#     energy = r.T@r
#     print(f"energy: {energy}\t dx: {np.linalg.norm(dx)}")
# print(X[0])


In [197]:
# rad_abs = np.sqrt(X[0])

# rads = np.zeros(V)
# for v_ind in range(V):
#     rads[v_ind] = (rad_abs ** 2 - np.dot(center_abs - v[v_ind], center_abs - v[v_ind])) / np.dot(center_abs - v[v_ind], normals[v_ind]) / 2
#     if (rads[v_ind] < 0):
#         print(v_ind)
# print(rads)

# centers = np.zeros((V, 3))
# for v_ind in range(V):
#     centers[v_ind] = v[v_ind] - normals[v_ind] * rads[v_ind]

# max_dif = abs(rads[0] - 1 / abs(h2[0]))
# min_dif = abs(rads[0] - 1 / abs(h2[0]))
# for v_ind in range(V):
#     max_dif = max(max_dif, abs(rads[v_ind] - 1 / abs(h2[v_ind])))
#     min_dif = max(min_dif, abs(rads[v_ind] - 1 / abs(h2[v_ind])))
# print(min_dif, max_dif)

In [198]:
# ps.init()
# ps.remove_all_structures()
 
# mesh = ps.register_surface_mesh("Mesh", v, f, smooth_shade=True)

# for _ in range(10):
#     i = np.random.randint(0, V-1)
#     c = centers[i] 
#     sphere = ps.register_point_cloud(f"sphere_c{i}", np.array([c]), enabled=True, color=(0,0,0), transparency=0.3)
#     sphere.set_radius(rads[i], relative=False)

# abs_sphere = ps.register_point_cloud(f"abs_sphere", np.array([center_abs]), enabled=True, color=(0,0,256), transparency=0.9)
# abs_sphere.set_radius(rad_abs, relative=False)

# ps.show()